In [1]:
!pip install plotly

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing
import os


In [3]:
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms as T
from torch import nn, optim
from torch.nn import functional as F
import torch.utils.data as data
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models

torch.manual_seed(42) # Set for reproducability

In [4]:
import timm
from timm.loss import LabelSmoothingCrossEntropy # Better than nn.CrossEntropyLoss

print(timm.__version__)
print(LabelSmoothingCrossEntropy)

0.6.7
<class 'timm.loss.cross_entropy.LabelSmoothingCrossEntropy'>


In [5]:
# remove warnings
import warnings
warnings.filterwarnings("ignore")

In [6]:
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline

In [7]:
import sys
from tqdm import tqdm
import time
import copy

In [8]:
# Constants
DATA_DIR = "../data/ship_100train_500val_200test"
DATA_SPLIT = [0.7, 0.2, 0.1]
BATCH_SIZE = 64
LR = 0.001
NUM_WORKERS = os.cpu_count()


In [9]:
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir + "/train")
    return all_data.classes

In [10]:
classes = get_classes(DATA_DIR)
print(classes, len(classes))

['ContainerShip', 'Cruise', 'Tanker', 'Warship'] 4


In [11]:
full_dataset = datasets.ImageFolder(DATA_DIR)
print(type(full_dataset))

<class 'torchvision.datasets.folder.ImageFolder'>


In [12]:
# def get_data_loader(data_dir, batch_size, ds_type, ds_split = [0.7, 0.2, 0.1]):
#     accepted_types = ('train', 'val', 'test')
#     assert ds_type in accepted_types, "Invalid Dataset Type"
    
#     full_dataset = datasets.ImageFolder(data_dir)
#     full_dataset_size = len(full_dataset)
    
#     assert len(ds_split) == 3, "ds_split does not have 3 sections"
#     assert round(sum(ds_split)) == 1, "ds_split ratio does not add to 1"
    
#     train_size = int(full_dataset_size * ds_split[0])
#     valid_test_size = full_dataset_size - train_size
    
#     train_dataset, val_test_dataset = data.random_split(full_dataset, [train_size, valid_test_size])
    
#     if ds_type == "train":
#         transform = T.Compose([
#             T.RandomHorizontalFlip(),
#             T.RandomVerticalFlip(),
#             T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.25),
#             T.Resize(256),
#             T.CenterCrop(224),
#             T.ToTensor(),
#             T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
#             T.RandomErasing(p=0.2, value='random'),
#         ])
#         train_dataset.dataset.transform = transform
#         train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS)
#         return train_loader, len(train_dataset)
    
#     else:
#         val_size = int(full_dataset_size * ds_split[1])
#         test_size = valid_test_size - val_size
        
#         transform = T.Compose([
#             T.Resize(256),
#             T.CenterCrop(224),
#             T.ToTensor(),
#             T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
#         ])
        
#         val_test_dataset.dataset.transform = transform
        
#         # Further split
#         val_dataset, test_dataset = data.random_split(val_test_dataset, [val_size, test_size])
        
#         if ds_type == "val":
#             val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS)
#             return val_loader, len(val_dataset)
            
#         elif ds_type == "test":
#             test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS)
#             return test_loader, len(test_dataset)

In [13]:
def get_data_loader(data_dir, batch_size, ds_type):
    if ds_type == "train":
        transform = T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.25),
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
            T.RandomErasing(p=0.2, value='random'),
        ])
        img_dataset = datasets.ImageFolder(os.path.join(data_dir, ds_type), transform=transform)
    
    else:
        transform = T.Compose([
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # imagenet means
        ])
        img_dataset = datasets.ImageFolder(os.path.join(data_dir, ds_type), transform=transform)
        
    data_loader = DataLoader(img_dataset, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS)
    
    return data_loader, len(img_dataset)
    
        
    

In [14]:
train_loader, train_size = get_data_loader(DATA_DIR, BATCH_SIZE, "train")
val_loader, val_size = get_data_loader(DATA_DIR, BATCH_SIZE, "val")
test_loader, test_size = get_data_loader(DATA_DIR, BATCH_SIZE, "test")

In [15]:
# train_loader, train_size = get_data_loader(DATA_DIR, BATCH_SIZE, "train", DATA_SPLIT)
# val_loader, val_size = get_data_loader(DATA_DIR, BATCH_SIZE, "val", DATA_SPLIT)
# test_loader, test_size = get_data_loader(DATA_DIR, BATCH_SIZE, "test", DATA_SPLIT)

In [16]:
dataloaders = {
    "train": train_loader,
    "val": val_loader,
    "test" : test_loader,
}

dataset_sizes = {
    "train": train_size,
    "val": val_size,
    "test": test_size,
}

In [17]:
print(f"Training Set has {train_size} data")
print(f"Val Set has {val_size} data")
print(f"Test Set has {test_size} data")
print(f"Train {len(train_loader)} batches, Val {len(val_loader)} batches, Test {len(test_loader)} batches")

Training Set has 400 data
Val Set has 2000 data
Test Set has 800 data
Train 7 batches, Val 32 batches, Test 13 batches


In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [19]:
model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=False)

Using cache found in /root/.cache/torch/hub/facebookresearch_deit_main


In [20]:
model

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate=none)
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (ls2): 

In [21]:
class LinearClassifier(nn.Module):
    """Linear layer to train on top of frozen features"""
    def __init__(self, dim, num_labels=1000):
        super(LinearClassifier, self).__init__()
        self.num_labels = num_labels
        self.linear = nn.Linear(dim, num_labels)
        self.linear.weight.data.normal_(mean=0.0, std=0.01)
        self.linear.bias.data.zero_()

    def forward(self, x):
        # flatten
        x = x.view(x.size(0), -1)

        # linear layer
        return self.linear(x)

# for param in model.parameters(): # freeze model
#     param.requires_grad = False


    
n_inputs = model.head.in_features

model.head = LinearClassifier(n_inputs, len(classes))

# model.head = nn.Sequential(
#     nn.Linear(n_inputs, 512),
#     nn.ReLU(),
#     nn.Dropout(0.3),
#     nn.Linear(512, len(classes)),
# )

model = model.to(device)
print(model)
    

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate=none)
        (drop1): Dropout(p=0.0, inplace=False)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (ls2): 

In [22]:
criterion = LabelSmoothingCrossEntropy()
criterion = criterion.to(device)
optimizer = optim.Adam(model.head.parameters(), lr=0.001)

In [23]:
# lr scheduler
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.97)


In [24]:
logs = df = pd.DataFrame(columns=['epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc'])
logs['epoch'] = df['epoch'].astype('int')

In [25]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs - 1}")
        print("-"*10)
        
        # Set model to correct mode for training/validation
        for phase in ['train', 'val']:
            if phase == "train":
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0.0
        
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
            
                with torch.set_grad_enabled(phase == "train"): # no autograd makes validation go faster
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) # used for accuracy
                    loss = criterion(outputs, labels)
                    
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == "train":
                scheduler.step() # step at end of epoch

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            if phase == "train":
                train_loss = epoch_loss
                train_acc = epoch_acc.item()
            else:
                val_loss = epoch_loss
                val_acc = epoch_acc.item()

            print(f"{phase} Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")

            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict()) # keep the best validation accuracy model
                
        logs.loc[len(logs.index)] = [epoch, train_loss, train_acc, val_loss, val_acc] 
        print()
        
    time_elapsed = time.time() - since
    print(f"Training completed in {time_elapsed // 60:.0f}m, {time_elapsed % 60:.0f}s")
    print(f"Best Val Acc: {best_acc:.4f}")
    
    model.load_state_dict(best_model_wts)
    return model,logs


In [ ]:
model_ft, logs = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=200)

Epoch 0/199
----------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:09<00:00,  1.38s/it]


train Loss: 1.4193, Acc: 0.2650


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:14<00:00,  2.24it/s]


val Loss: 1.3962, Acc: 0.2815

Epoch 1/199
----------


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:14<00:00,  2.21it/s]


val Loss: 1.2782, Acc: 0.4315

Epoch 34/199
----------


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:08<00:00,  1.25s/it]


train Loss: 1.1649, Acc: 0.5125


  0%|                                                                                                                                              | 0/32 [00:00<?, ?it/s]

In [ ]:
logs

In [ ]:
logs.to_csv('deitbase_logs_100train.csv', index=False)

In [ ]:
fig = go.Figure()

# Add Traces
fig.add_trace(
    go.Scatter(x=logs['epoch'],
               y=logs['train_loss'],
               name='Train Loss',
               line=dict(color='firebrick', width=2)))

fig.add_trace(
    go.Scatter(x=logs['epoch'],
               y=logs['train_acc'],
               name='Train Acc',
               line=dict(color='green', width=2)))

fig.add_trace(
    go.Scatter(x=logs['epoch'],
               y=logs['val_loss'],
               name='Val Loss',
               line=dict(color='blue', width=2)))

fig.add_trace(
    go.Scatter(x=logs['epoch'],
               y=logs['val_acc'],
               name='Val Acc',
               line=dict(color='blue', width=2)))

# Edit the layout
fig.update_layout(title='Pretext Training',
                   xaxis_title='Epochs',
                   yaxis_title='Loss')

#Add Buttons
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            x=0.7,
            y=1.22,
            buttons=list([
                dict(label="DeiT-Tiny",
                     method="update",
                     args=[{"visible": [True, True, True, True]},
                           {"title": "Pretext Analysis",}
                        ]
                    ),
                dict(label="Loss",
                     method="update",
                     args=[{"visible": [True, False, True, False]},
                           {"title": "Train:Val Loss",}
                        ]
                    ),
                dict(label="Acc",
                     method="update",
                     args=[
                           {"visible": [False, True, False, True]},
                           {"title": "Train:Val Acc",}
                        ]
                    ),
            ]),
        )
    ])

# Set title
fig.update_layout(
    title_text="DeiT-Tiny Supervised",
    xaxis_domain=[0.05, 1.0],
    xaxis_title="Epochs",
    yaxis_title="Loss/Acc",
)

fig.show()


In [ ]:
test_loss = 0.0
class_correct = list(0 for i in range(len(classes)))
class_total = list(0 for i in range(len(classes)))
model.to(device)
model.eval()

for data, target in tqdm(test_loader, leave=True):
    data, target = data.to(device), target.to(device)
    with torch.no_grad(): # turn off autograd for faster testing
        output = model(data)
        loss = criterion(output, target)
    test_loss = loss.item() * data.size(0)
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    if len(target) == BATCH_SIZE:
        for i in range(BATCH_SIZE):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
            
test_loss = test_loss / test_size
print('Test Loss: {:.4f}'.format(test_loss))
for i in range(len(classes)):
    if class_total[i] > 0:
        print(f"Test Accuracy of {classes[i]}: {100*class_correct[i]/class_total[i]:.2f}% ({class_correct[i]}/{class_total[i]})")
    else:
        print("Test accuracy of %5s: NA" % (classes[i]))

print(f"Test Accuracy of {100*np.sum(class_correct)/np.sum(class_total):.2f}% ({np.sum(class_correct)}/{np.sum(class_total)})")